In [33]:
def create_netcdf(inyear, outyear, nmonths, pressin, lonin, latin,tracer):

    from netcdftime import utime, JulianDayFromDate,DateFromJulianDay, date2index
    #from netcdftime import datetime as datetimex,num2date
    from netCDF4 import Dataset
    import time
    from netCDF4 import num2date, date2num
    # create a series of dates for the netcdf file in times.
    from datetime import datetime, timedelta
    from netCDF4 import num2date, date2num

    # create a series of dates for the netcdf file in times.
    from datetime import datetime, timedelta

    field_o = np.load('ordonez_res'+str(inyear)+'.npy')
    field_l = np.load('liang_res'+str(inyear)+'.npy')
    field_z = np.load('ziska_res'+str(inyear)+'.npy')
    nmonths1, nlevs1, ny1, nx1 = np.shape(field_o)
    dataset = Dataset('N96_L85_CHBr3_'+str(outyear)+'_tracer'+str(tracer)+'.nc', 'w', format='NETCDF4')

    

    # Global Attributes
    dataset.description = 'TEST NETCDF-CF COMPLIANT SCRIPT'
    dataset.history = 'Created ' + time.ctime(time.time())
    dataset.source = ''
    dataset.Conventions = 'CF-1.0'
    dataset.standard_name_vocabulary='CF-1.0'

    time  = dataset.createDimension('time', None)
    time  = dataset.createVariable( 'time', 'f8', ('time',)) 
    time.standard_name = 'time'
    time.long_name = 'time'
    time.units = 'days since 1991-09-01 00:00:00'
    time.calendar = '360_day'
    time.axis = 'T'
    
    dates = []
    times = []
    for n in range (0,12):
        # nasty fix for the fact that datetime.timedelta only works on 365 day calendar
        new_date =  date2num(datetime(inyear,1,16),units=time.units, calendar=time.calendar)+n*30
        dates.append(num2date(new_date, units=time.units, calendar=time.calendar))
        
    time[:] = date2num(dates,units=time.units,calendar=time.calendar)
    
    # Create HDF5 *format*, classic *model*

    height = dataset.createDimension('pressure', nplevs)
    height = dataset.createVariable('pressure',  np.int32,   ('pressure',))
    height.units = 'mbar'
    height.axis='Z'
    height.positive='up'
    height.standard_name = 'pressure_levels'

    lat   = dataset.createDimension('lat',  ny1)
    lat   = dataset.createVariable('lat',   np.float32, ('lat',))
    lat.standard_name = 'latitude'
    lat.units = 'degrees_north'
    lat.cartesian_axis = "Y"
    lat.axis = 'Y'
    lat.actual_range = -90.0, 90.0

    lon   = dataset.createDimension('lon', nx1)
    lon   = dataset.createVariable('lon',  np.float32, ('lon',))
    lon.standard_name = 'longitude'
    lon.units = 'degrees_east'
    lon.cartesian_axis = "X"
    lon.axis = "X"
    lat.actual_range = 0.0, 360.0

    if tracer == 1:
        #liang
        tracer1 = dataset.createVariable('Tracer1_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
        tracer1.units = 'kg kg-1'
        tracer1.standard_name='CHBr3 mass mixing ratio'
        tracer1[:,:,:,:] = field_l[:,:,:,:]
    elif tracer ==2:
        #ordonez
        tracer2 = dataset.createVariable('Tracer2_mmr', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
        tracer2.units = 'kg kg-1'
        tracer2.standard_name='CH2Br2 mass mixing ratio'
        tracer2[:,:,:,:] = field_o[:,:,:,:]    
    elif tracer ==3:
        tracer3 = dataset.createVariable('Tracer3_mmr', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
        tracer3.units = 'kg kg-1'
        tracer3.standard_name='CHBr3 mass mixing ratio'
        tracer3[:,:,:,:] = field_z[:,:,:,:]
    
#     tracer1 = dataset.createVariable('Tracer1_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
#     tracer1.units = 'kg kg-1'
#     tracer1.standard_name='CHBr3 mass mixing ratio'

#     tracer2 = dataset.createVariable('Tracer2_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
#     tracer2.units = 'kg kg-1'
#     tracer2.standard_name='CHBr3 mass mixing ratio'

#     tracer3 = dataset.createVariable('Tracer3_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
#     tracer3.units = 'kg kg-1'
#     tracer3.standard_name='CHBr3 mass mixing ratio'

    lon[:]   = lonin[:]
    lat[:]   = latin[:]
    height[:] = pressin[:]

#     tracer1[:,:,:,:] = field_l[:,:,:,:]
#     tracer2[:,:,:,:] = field_o[:,:,:,:]
#     tracer3[:,:,:,:] = field_z[:,:,:,:]

    print ('success')
    print 'time values (in units %s): ' % time.units+'\n',time[:]
    dataset.close()

    return outyear

In [34]:
import subprocess as sb
import numpy as np
import netCDF4

press = np.array([1000,950,900,850,700,500,400,300,250,200,150,100,70,50,30,20,10])*100
nplevs= len(press)

filename='/Users/ptg21/data/projects/TRANSCOM/N96_ancil_creation/xkawa.p.avg2000.nc'
ncfile2     = netCDF4.Dataset(filename)

lon2        = ncfile2.variables['longitude'][:]
lat2        = ncfile2.variables['latitude'][:]
pressure    = ncfile2.variables['p'][:]
times2      = ncfile2.variables['time']
inyears = [[1994, 1994], [1995, 1995], [1996, 1996], [1997, 1997], [1998, 1998], [1999, 1999],[2000, 2000]]
outyears = inyears

for outyear in outyears:
    filename_to_remove =  'N96_L85_CHBr3_'+str(yearpair[1])+'.nc'
    print ('removing '+ filename_to_remove)
    sb.Popen(['rm', '-Rf', filename_to_remove]).wait()

    
for tracer in [1,2,3]:
    for yearpair in outyears:
        print yearpair
        created_for = create_netcdf(inyear = yearpair[0], outyear=yearpair[1], nmonths = times2,\
                      pressin = press/100. , lonin = lon2, latin = lat2, tracer = tracer)
        print
        print 'netcdf created for', created_for
        print
        

removing N96_L85_CHBr3_2000.nc
removing N96_L85_CHBr3_2000.nc
removing N96_L85_CHBr3_2000.nc
removing N96_L85_CHBr3_2000.nc
removing N96_L85_CHBr3_2000.nc
removing N96_L85_CHBr3_2000.nc
removing N96_L85_CHBr3_2000.nc
[1994, 1994]
success
time values (in units days since 1991-09-01 00:00:00): 
[  855.   885.   915.   945.   975.  1005.  1035.  1065.  1095.  1125.
  1155.  1185.]

netcdf created for 1994

[1995, 1995]
success
time values (in units days since 1991-09-01 00:00:00): 
[ 1215.  1245.  1275.  1305.  1335.  1365.  1395.  1425.  1455.  1485.
  1515.  1545.]

netcdf created for 1995

[1996, 1996]
success
time values (in units days since 1991-09-01 00:00:00): 
[ 1575.  1605.  1635.  1665.  1695.  1725.  1755.  1785.  1815.  1845.
  1875.  1905.]

netcdf created for 1996

[1997, 1997]
success
time values (in units days since 1991-09-01 00:00:00): 
[ 1935.  1965.  1995.  2025.  2055.  2085.  2115.  2145.  2175.  2205.
  2235.  2265.]

netcdf created for 1997

[1998, 1998]
success
ti

In [36]:
%%bash
rm -Rf N96_L85_CHBr3_tracer1.nc
ncrcat N96_L85_CHBr3_{1994,1995,1996,1997,1998,1999,2000}_tracer1.nc N96_L85_CHBr3_tracer1.nc

rm -Rf N96_L85_CHBr3_tracer2.nc
ncrcat N96_L85_CHBr3_{1994,1995,1996,1997,1998,1999,2000}_tracer2.nc N96_L85_CHBr3_tracer2.nc

rm -Rf N96_L85_CHBr3_tracer3.nc
ncrcat N96_L85_CHBr3_{1994,1995,1996,1997,1998,1999,2000}_tracer3.nc N96_L85_CHBr3_tracer3.nc


In [37]:
%%bash 

ls

Interp_onto_p_levels.ipynb
N48_L60_CHBr3_1994_tracer1.nc
N48_L60_CHBr3_1994_tracer2.nc
N48_L60_CHBr3_1994_tracer3.nc
N48_L60_CHBr3_1995_tracer1.nc
N48_L60_CHBr3_1995_tracer2.nc
N48_L60_CHBr3_1995_tracer3.nc
N48_L60_CHBr3_1996_tracer1.nc
N48_L60_CHBr3_1996_tracer2.nc
N48_L60_CHBr3_1996_tracer3.nc
N48_L60_CHBr3_1997_tracer1.nc
N48_L60_CHBr3_1997_tracer2.nc
N48_L60_CHBr3_1997_tracer3.nc
N48_L60_CHBr3_1998_tracer1.nc
N48_L60_CHBr3_1998_tracer2.nc
N48_L60_CHBr3_1998_tracer3.nc
N48_L60_CHBr3_1999_tracer1.nc
N48_L60_CHBr3_1999_tracer2.nc
N48_L60_CHBr3_1999_tracer3.nc
N48_L60_CHBr3_2000_tracer1.nc
N48_L60_CHBr3_2000_tracer2.nc
N48_L60_CHBr3_2000_tracer3.nc
N48_L60_ch2br2_1994_tracer1.nc
N48_L60_ch2br2_1994_tracer2.nc
N48_L60_ch2br2_1994_tracer3.nc
N48_L60_ch2br2_1995_tracer1.nc
N48_L60_ch2br2_1995_tracer2.nc
N48_L60_ch2br2_1995_tracer3.nc
N48_L60_ch2br2_1996_tracer1.nc
N48_L60_ch2br2_1996_tracer2.nc
N48_L60_ch2br2_1996_tracer3.nc
N48_L60_ch2br2_1997_tracer1.nc
N48_L60_ch2br2_1997_tracer2.nc
N4

In [38]:
%%bash
cp N96_L85_CHBr3_tracer1.nc mm.UMUKCA.UCam.1.nc.N96L85
cp N96_L85_CHBr3_tracer2.nc mm.UMUKCA.UCam.2.nc.N96L85
cp N96_L85_CHBr3_tracer3.nc mm.UMUKCA.UCam.3.nc.N96L85